In [1]:
import matplotlib.pyplot as pt
import numpy as np
import pandas as pd

In [2]:
# przygotowanie danych testowych
supported_langs = ["eng", "deu"]
csv_file = pd.read_csv('sentences.csv', on_bad_lines='skip', sep='\t', index_col=0, names=["lang", "sentence" ])
supported_languages_mask = csv_file['lang'].isin(supported_langs)
dataset = csv_file[supported_languages_mask]

In [3]:
results = pd.DataFrame(columns=["lang","sentence"])
for l in supported_langs:
    ds = dataset[dataset["lang"] == l].sample(50)
    results = pd.concat([results, ds])
display(results)

,lang,sentence
10296801,eng,Ziri knows Rima's name now.
27086,eng,The hermit lived in a wooden hut.
3732221,eng,Tom wouldn't tell Mary why he had quit his job.
6653767,eng,Tom said that he hated being treated like a ch...
8171509,eng,Tom bragged to his friend about it.
...,...,...
7809377,deu,"Wenn du Tom zeigst, wie das geht, kriegt er es..."
8548912,deu,"Ein leichtes Essen ist es, was wir brauchen."
3851595,deu,Ich bin fasziniert von ihrer Schönheit.
999147,deu,Er stellt oft dumme Fragen.


In [4]:

from collections import Counter
def get_trigrams(sentence: str):
    trigrams = []
    for i, c in enumerate(sentence):
        trigrams.append(sentence[i:(i + 3)])
    return trigrams

In [5]:
def create_matrix(dataset):
    most_common_trigrams = set()
    most_common_trigrams_by_lang = {}
    for lang in supported_langs:
        trigrams = Counter()
        series = dataset[dataset["lang"] == lang]["sentence"].head(20)
        for sentence in series:
            tri = get_trigrams(sentence)
            trigrams.update(tri)
        mc = trigrams.most_common(20)
        most_common_trigrams.update([v[0] for v in mc])
        most_common_trigrams_by_lang[lang] = mc
    return most_common_trigrams, most_common_trigrams_by_lang

In [6]:
most_common_trigrams, most_common_trigrams_by_lang = create_matrix(results)

In [8]:
occurences = pd.DataFrame(columns=most_common_trigrams)
for index, row in results.iterrows():
    trigrams = get_trigrams(row["sentence"])
    for trigram in trigrams:
        occurences.append(trigram)

C:\Users\48797\AppData\Local\Temp\ipykernel_16544\4023284287.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  occurences.append(trigram)


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid